In [1]:
import os
import datetime
import numpy as np

from src.default_paths import path_root
from src.utils import list_dir, get_best_clmbr_model

In [2]:
path_models = os.path.join(path_root, "data/clmbr_models")

In [3]:
all_paths = {}
for model in ["clmbr_sk", "clmbr_stanford_cp", "clmbr_stanford_tapt"]:
    
    all_paths[model] = os.path.join(
        path_models, 
        model, 
        get_best_clmbr_model(os.path.join(path_models, model))
    )

In [4]:
def get_loss_from_line(line):
    return line.split("'loss': ")[1].split(",")[0]

def get_datetime_from_line(line):
    str_datetime = line.split("[")[0].split(",")[0]
    str_date, str_ts = str_datetime.split(" ")
    y, m, d = [int(x) for x in str_date.split('-')]
    h, mi, s = [int(x) for x in str_ts.split(':')] 
    return datetime.datetime(y,m,d,h,mi,s)

In [5]:
losses = {}
for model, path_model in all_paths.items():
    
    losses[model] = {
        "dev_loss": [],
        "dev_loss_datetime": []
    }
    
    path_log = os.path.join(path_model, "clmbr_model", "log")
    
    with open(path_log, "r") as f:
        for line in f:
            if "[Step 0]" in line:
                losses[model]["starting_datetime"] = get_datetime_from_line(line)
            if "Dev loss" in line:
                losses[model]["dev_loss"].append(get_loss_from_line(line))
                losses[model]["dev_loss_datetime"].append(get_datetime_from_line(line))
                
    index_best_model = np.where(
        np.asarray(losses[model]["dev_loss"])==min(losses[model]["dev_loss"])
    )[0][0]
    
    time_to_best_model = round(
        (losses[model]["dev_loss_datetime"][index_best_model] - losses[model]["starting_datetime"]).total_seconds()/3600,
        2
    )
    
    print(f"{model=}, {time_to_best_model=} hours")

model='clmbr_sk', time_to_best_model=9.86 hours
model='clmbr_stanford_cp', time_to_best_model=1.46 hours
model='clmbr_stanford_tapt', time_to_best_model=0.99 hours
